In [1]:
import pandas as pd
import numpy as np
import re
import glob
from google.cloud import bigquery_storage
import os
from google.cloud import bigquery

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Load the data with selected columns
recent = pd.read_csv('/Users/reppmazc/Documents/IRONHACK/quests/final_project/news_data.csv',
                     usecols=['url', 'publishedAt', 'content'])

# Rename the 'publishedAt' column to 'datetime'
recent = recent.rename(columns={'publishedAt': 'datetime'})

# Define a function to format the datetime string to YYYYMMDDHHMMSS
def format_datetime_string(dt_string):
    # Remove timezone offset if it exists (e.g., "+00:00")
    dt_string = dt_string.split('+')[0]
    # Remove 'T' if present and join parts to get the desired format
    dt_string = dt_string.replace('T', '').replace('-', '').replace(':', '').replace(' ', '')
    return dt_string

# Apply the function to the 'datetime' column
recent['datetime'] = recent['datetime'].astype(str).apply(format_datetime_string)

In [3]:
# concatenating files -> all follow the same structure but have different rows
# Define the path to your files
file_path = '/Users/reppmazc/Documents/IRONHACK/quests/final_project/scraped_content_*.csv'

# Use glob to get all file paths matching the pattern
all_files = glob.glob(file_path)

# Read each file and concatenate them into a single DataFrame
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [4]:
df.shape

(67000, 2)

In [5]:
df.to_csv('/Users/reppmazc/Documents/IRONHACK/quests/final_project/all_articles_notcleaned.csv')

In [6]:
# Rename specific columns
df = df.rename(columns={'Content': 'content',
                        'DocumentIdentifier': 'url'})

In [7]:
df.shape

(67000, 2)

In [8]:
df = pd.concat([df, recent], ignore_index=True)

In [9]:
# remove duplicate rows going by the hyperlink in the 'DocumentIdentifier' column
df = df.drop_duplicates(subset='url', keep='first')

In [10]:
df.shape

(57000, 3)

In [11]:
# remove rows where the 'DocumentIdentifier' column contains dumb or non-german (speaking) outlets
exclude_list = [
    'www.urlaubspiraten.de', 'www.kino-zeit', 'www.macwelt', 'www.autobild', 'www.wrdw', 
    '.orf.at', 'diepresse.com', 'www.dw.com/es', 'www.dw.com/uk', 'www.wrdw.com/2024', 
    'www.dw.com/ru', 'www.dw.com/en', 'www.dw.com/pt-br', 'www.dw.com/tr', 'www.dw.com/sw', 
    'www.dw.com/fr', 'www.dw.com/bg', 'www.dw.com/sr', 'www.dw.com/bn', 'www.dw.com/mk', 
    'www.dw.com/pl', 'www.dw.com/ar', 'www.dw.com/ro', 'www.dw.com/hr', 'www.dw.com/id']

# create a regex pattern by joining the list items with '|'
pattern = '|'.join(exclude_list)

# filter out rows where 'DocumentIdentifier' contains any of the substrings
df = df[~df['url'].str.contains(pattern, na=False)]

In [12]:
df.shape

(45117, 3)

In [13]:
# replace text in 'content' column by nan that contains freaky signs like these:
freaky_pattern = r'Ã¼Ã¤Ã¶ÃŸâ¬Ã'

# Replace rows in 'content' column with NaN where 'freaky' characters are found
df['content'] = df['content'].apply(lambda x: np.nan if pd.notna(x) and bool(re.search(freaky_pattern, x)) else x)

In [14]:
df.shape

(45117, 3)

In [15]:
# checking if text is german
# common German words
common_german_words = [
    "und", "der", "das", "ist", "zu", "mit", "von", "auf", "für", "den", "im", "ein", "nicht",
    "eine", "als", "auch", "aber", "wie", "es", "am", "aus", "bei", "dass", "oder", "so", "wenn",
    "werden", "wir", "hat", "sich", "dem", "des", "noch", "nur", "kann", "um", "ja", "mehr"]

# Higher threshold for minimum word matches
threshold = 5

# Check if a text is likely in German
def is_german(text):
    # Count occurrences of common German words
    word_count = sum(word in text.lower() for word in common_german_words)
    # Return NaN if it doesn't meet the threshold, otherwise return the text
    return text if word_count >= threshold else np.nan

# Apply function to the 'content' column
df['content'] = df['content'].apply(lambda x: is_german(x) if isinstance(x, str) else np.nan)

In [16]:
df.shape

(45117, 3)

In [17]:
# removing strings that are part of the article content of some cells
# Define the non-article snippets and phrases to trim
non_article_snippets = [
    'Hauptnavigation: Nutzen Sie die Tabulatortaste, um durch die Menüpunkte zu navigieren. Öffnen Sie Untermenüs mit der Leertaste. Schließen Sie Untermenüs mit der Escape-Taste. Hauptnavigation: Nutzen Sie die Tabulatortaste, um durch die Menüpunkte zu navigieren. Öffnen Sie Untermenüs mit der Leertaste.',
    'Lesen Sie mehr zum Thema In anspruchsvollen Berufsfeldern im Stellenmarkt der SZ. Sie möchten die digitalen Produkte der SZ mit uns weiterentwickeln? Bewerben Sie sich jetzt!Jobs bei der SZ Digitale Medien Gutscheine:',
    'öffnet in neuem Tab oder Fenster',
    'Danke, dass Sie ZEIT ONLINE nutzen.',
    'Melden Sie sich jetzt mit Ihrem bestehenden Account an oder testen Sie unser digitales Abo mit Zugang zu allen Artikeln. Erscheinungsbild Die Zusammenfassung für diesen Artikel kann leider momentan nicht angezeigt werden.',
    'ZEIT ONLINE hat diese Meldung redaktionell nicht bearbeitet. Sie wurde automatisch von der Deutschen Presse-Agentur (dpa) übernommen.',
    'Kommentar | ',
    'Berlin (dpa/bb). ',
    '+++ ',
    '++',
    '© dpa-infocom, ',
    'Drucken Teilen',
    'Nicht verpassen: Alles rund ums Thema Job & Beruf finden Sie im Karriere-Newsletter unseres Partners Merkur.de.',
    'Erstellt durch: ',
    '0 Weniger als eine Minute',
    '► ',
    '© ',
    '© dpa/',
    '© REUTERS/',
    '© Getty Images',
    '© IMAGO/',
    '© imago images/',
    '© imago/',
    'Lesen Sie auch',
    '© Berliner Feuerwehr ',
    '© privat ',
    'Kopiere den aktuellen Link ',
    'DIE ZEIT: ']

phrases_to_trim_after = [
    'Die WELT als ePaper: Die vollständige Ausgabe steht Ihnen bereits am Vorabend zur Verfügung – so sind Sie immer hochaktuell informiert. Weitere Informationen http://epaper.welt.de Der Kurz-Link dieses Artikels lautet:',
    'Hier können Sie interessante Artikel speichern',
    'Aktuelle Nachrichten und Hintergründe aus Politik, Wirtschaft und Sport aus Berlin, Deutschland und der Welt.']

# Step 1: Remove specified snippets from within the content
def remove_snippets(text):
    if pd.notna(text):  # Ensure text is not NaN
        for snippet in non_article_snippets:
            text = text.replace(snippet, '')  # Remove each snippet if it appears
    return text

df['content'] = df['content'].apply(remove_snippets)

# Step 2: Trim content after any of the specified phrases
def trim_content(text):
    if pd.notna(text):  # Ensure text is not NaN
        for phrase in phrases_to_trim_after:
            if phrase in text:
                return text.split(phrase)[0]  # Keep only the part before the phrase
    return text

df['content'] = df['content'].apply(trim_content)

0    Im Streit über die sogenannte Gendersprache ha...
1     Die Vorsitzenden der Grünen Jugend, Svenja Ap...
2    Krieg in Nahost : Israels heikle Optionen\n\nE...
3     Carsten Berger  Das Angebot kam plötzlich: 20...
5     Bundesinnenministerin Faeser hat das vom Bund...
Name: content, dtype: object


In [18]:
df.shape

(45117, 3)

In [19]:
def clean_text(text):
    if isinstance(text, str):  # Ensure text is a string
        text = re.sub(r"http\S+", "", text)  # Remove URLs in general
        text = re.sub(r"©\s?\d+", "", text)  # Remove copyright symbols
        text = re.sub(r'\n+', ' ', text)  # Replace newlines with spaces
        text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
        return text
    return text  # Return NaN or non-string content as-is

# Apply the cleaning function to each article
df['content'] = df['content'].apply(clean_text)

In [20]:
df.shape

(45117, 3)

In [21]:
# nan in cells with only whitespace
df['content'] = df['content'].apply(lambda x: np.nan if isinstance(x, str) and x.strip() == '' else x)

In [22]:
df.shape

(45117, 3)

In [24]:
columns_to_check = [col for col in df.columns if col != 'datetime']
df = df.dropna(subset=columns_to_check)

In [25]:
df.shape

(39677, 3)

In [26]:
# create new source column from hyperlink in 'DocumentIdentifier' column as string (source is between the first . and the second . in the hyperlink)
df['source'] = df['url'].str.extract(r'\.(\w+)\.')

In [27]:
df.shape

(39677, 4)

In [28]:
# removed any non-approved sources
allowed_sources = ['news', 'merkur', 'finanznachrichten', 'gruenderszene', 'hna', 'fnp', 'kreiszeitung', 'stern',
                   'giessener-allgemeine', 'sueddeutsche', 'presseportal', 'gmx', 'op-online', 'tz', 'freenet',
                   'giessener-anzeiger', 'come-on', 'n-tv', 'zeit', 'wallstreet-online', 'az-online', 'bild', 'rheinpfalz',
                   'tagesspiegel', 'ad-hoc-news', 'badische-zeitung', 'faz', 'volksstimme', 'mannheim24', 'leinetal24',
                   'schwarzwaelder-bote', 'shz', 'wn', 'fuldaerzeitung', 'ndr', 'ruhrnachrichten', 'stuttgarter-nachrichten',
                   'donaukurier', 'nachrichten-heute', 'schwaebische', 'webnachrichten', 'hanauer', 'mittelbayerische', 'pnp',
                   'rp-online', 'wz', 'nwzonline', 'lomazoma', 't-online', 'stuttgarter-zeitung', 'tag24', 'gn-online', 'onetz',
                   'ruhr24', 'waz', 'welt', 'mz', 'saarbruecker-zeitung', 'dattelner-morgenpost', 'nn', 'aachener-zeitung',
                   'bo', 'express', 'volksfreund', 'nw', 'noz', 'dorstenerzeitung', 'halternerzeitung', 'hellwegeranzeiger',
                   'pz-news', 'augsburger-allgemeine', 'krzbb', 'web', 'morgenweb', 'wp', 'rnz', 'nordsee-zeitung',
                   'mannheimer-morgen', 'swp', 'chip', 'tagesschau', 'nordkurier', 'abendblatt', 'nordbayern', 'zdf',
                   'bnn', 'kn-online', 'abendzeitung-muenchen', 'spiegel', 'westfalen-blatt', 'insuedthueringen',
                   'echo-online', 'mainpost', 'sport', 'frankenpost', 'lz', 'ka-news', 'weser-kurier', 'zvw', 'ln-online',
                   'mdr', 'allgaeuer-zeitung', 'infranken', 'general-anzeiger-bonn', 'hochheimer-zeitung', 'otz',
                   'wormser-zeitung', 'mt', 'derpatriot', 'wr', 'morgenpost', 'wochenblatt-reporter', 'gea',
                   'goettinger-tageblatt', 'blick', 'neuepresse', 'on-online', 'waz-online', 'nrz', 'wired-de', 'fnweb',
                   'kurier', 'dewezet', 'muensterschezeitung', 'mittelhessen', 'bergstraesser-anzeiger', 'paz-online',
                   'schwetzinger-zeitung', 'braunschweiger-zeitung', 'ga', 'oz-online', 'nh24', 'lvz', 'op-marburg',
                   'cannstatter-zeitung', 'derstandard', 'wirtschafts-woche', 'solinger-tageblatt', 'derwesten', 'swr',
                   'mopo', 'antenneniederrhein', 'berliner-zeitung', 'freiepresse', 'deutschlandfunk', 'maz-online',
                   'handelsblatt', 'stimme', 'radioerft', 'thueringer-allgemeine', 'tlz', 'esslinger-zeitung', 'pressebox',
                   'radiokoeln', 'ikz-online', 'radioenneperuhr', 'radio-bamberg', 'radioeins', 'gala', 'radio-plassenburg',
                   'lampertheimer-zeitung', 'radio912', 'rga', 'rnd', 'sn-online', 'dnn', 'radioeuskirchen', 'bbv-net',
                   'sat1regional', 'taz', 'idowa', 'lippewelle', 'radiokiepenkerl', 'hellwegradio', 'radiooberhausen',
                   'soester-anzeiger', 'bunte', 'moin', 'rtl', 'antennemuenster', 'businessinsider', 'news38', 'main-netz',
                   'harzkurier', 'radiomk', 'hessenschau', 'presse-board', 'radio901', 'iz', 'moz', 'br', 'lkz', 'sport1',
                   'thueringen24', 'epochtimes', 'haz', 'radiokw', 'lr-online', 'prosieben', 'np-coburg', 'obermain',
                   'presse-augsburg', 'lokalkompass', 'osthessen-news', 'unternehmen-heute', 'news894', 'ejz', 'city-news',
                   'ffh', 'idw-online', 'radiobonn', 'radiomuelheim', 'radiowmw', 'goslarsche', 'radiorst', 'wdr',
                   'hersfelder-zeitung', 'schlaunews', 'brigitte', 'heise', 'fr', 'radiohagen', 'radiovest', 'watson',
                   'berliner-kurier', 'regio-journal', 'sportschau', 'berliner-abendblatt', 'tageblatt',
                   'oldenburger-onlinezeitung', 'wiesbadener-kurier', 'bietigheimerzeitung', 'rosenheim24', 'deraktionaer',
                   'familie', 'rbb24', 'sat1', 'all-in', 'deutschlandfunkkultur', 'presseschleuder', 'saechsische', 'vip',
                   'azonline', 'jungewelt', 'spox', 'nd-aktuell', 'schwaebische-post', 'allgemeine-zeitung', 'rollingpin',
                   'aerztezeitung', 'juedische-allgemeine', 'innsalzach24', 'buerstaedter-zeitung', 'mein-mmo', 'notebookcheck',
                   'winfuture', 'borkenerzeitung', 'dubisthalle', 'moritz', 'openpr', 'presse-nachrichten', 'chiemgau24', 'dw',
                   'echo24', 'hasepost', 'inar', 'kreisbote', 'artikel-presse', 'hildesheimer-allgemeine', 'presseradar',
                   'radio38', 'shk-journal', 'turi2', 'unsertirol24', 'fcbinside', 'siegener-zeitung', 'topagrar',
                   'xn--brgersagt-q9a', 'focus', 'pressnetwork', 'stadt-bremerhaven', 't3n', 'deutsche-wirtschafts-nachrichten',
                   'die-neue-welle', 'wetterauer-zeitung', 'contentmanager', 'journalistenwatch', 'manager-magazin', 'ovb-online',
                   'rollingstone', 'sol', 'kreis-anzeiger', 'wiesentbote', 'bgland24', 'prisma', 'capital', 'freitag', 'inside-digital',
                   'kabeleins', 'nord24', 'ramasuri', 'sauerlandkurier', 'lokalo', 'telepolis', 'tichyseinblick', 'usinger-anzeiger',
                   'euractiv', 'geo', 'karlsruhe-insider', 'lebensmittelzeitung', 'ostsee-zeitung', 'pressetext', 'weltjournal',
                   'futurezone', 'fvw', 'gamereactor', 'gmuender-tagespost', 'hl-live', 'news8', 'unitednetworker', '24rhein',
                   'jungefreiheit', 'mmnews', 'pfalz-express', 'pressecontrol', 'wa', 'ak-kurier', 'haufe', 'katholisch', 'ludwigsburg24',
                   'ukrinform', 'viply', 'ahgz', 'appgefahren', 'berlin', 'daserste', 'fuldainfo', 'hallo-muenchen', 'hz', 'lifepr',
                   'neckaralblive', 'ok-magazin', 'play3', 'macerkopf', 'wlz-online', 'ingame', 'l-iz', 'lauterbacher-anzeiger', 'mrn-news',
                   'nrwz', 'harpersbazaar', 'hightechbox', 'mainfranken24', 'radioleverkusen', 'report-k', 'wochenkurier', 'irw-press',
                   'monopol-magazin', 'niederlausitz-aktuell', 'ntz', 'sonntagsblatt', 'spektrum', 'cicero', 'oekotest', 'vogel', 'buzzfeed',
                   'die-tagespost', 'donau3fm', 'e110', 'medical-tribune', 'rdl', 'rundblick-unna', 'ww-kurier', 'aero', 'monstersandcritics',
                   'nr-kurier', 'rotenburger-rundschau', 'swr3', 'wz-net', 'asscompact', 'brn-ag', 'deichstube', 'esut', 'kraichgau',
                   'ludwigshafen24', 'mittelrhein-tageblatt', 'oberhessische-zeitung', 'scinexx', 'szbz', 'teltarif', 'travelbook',
                   'wirtschaft-in-sachsen', 'ardmediathek', 'areadvd', 'baunetz', 'behoerden-spiegel', 'bergkamen-infoblog', 'harburg-aktuell',
                   'lto', 'raptastisch', 'reitschuster', 'remszeitung', 'report24', 'riffreporter', 'springermedizin', 'vdi-nachrichten',
                   'bmvg', 'cio', 'dawo-dresden', 'dbwv', 'dzonline', 'elektro', 'gaeubote', 'jesus', 'landeswelle', 'mena-watch',
                   'nachrichten-muenchen', 'netzpolitik', 'nordstadtblogger', 'nuernberger-blatt', 'politikstube', 'queer', 'theeuropean',
                   'volcanodiscovery', 'werra-rundschau', 'wochenanzeiger', 'absatzwirtschaft', 'bernau-live', 'buffed', 'business-on',
                   'charivari', 'epd', 'gandersheimer-kreisblatt', 'garbsen-city-news', 'intelligent-investors', 'invidis', 'kma-online',
                   'marler-zeitung', 'migazin', 'norderneyer-badezeitung', 'paz', 'radio-rur', 'regenbogen', 'rtf1', 'stylebook',
                   'taunus-nachrichten', 'utopia', 'vogue', 'ad-magazin', 'bayreuth', 'blog-der-republik', 'bz-berlin', 'einbecker-morgenpost',
                   'esanum', 'hcm-magazin', 'heidelberg24', 'helmholtz', 'honnef-heute', 'ingolstadt-today', 'israelnetz', 'iww',
                   'kevelaerer-blatt', 'klimareporter', 'muehlacker-tagblatt', 'muensterlandzeitung', 'onvista', 'opposition24',
                   'politplatschquatsch', 'rbb-online', 'regensburger-nachrichten', 'rheiderland', 'rohmert-medien', 'solarserver',
                   'st-georg', 'tip-berlin', 'unternehmeredition', 'alfelder-zeitung', 'b2b-wirtschaft', 'bba-online', 'berlin-live',
                   'business-echo', 'darmstadtnews', 'desired', 'dieunbestechlichen', 'docma', 'edison', 'electrive', 'elle',
                   'ems-vechte-surfer', 'emtb-news', 'glamour', 'heide-kurier', 'hr-fernsehen', 'kiel-magazin', 'lokal-anzeiger-erkrath',
                   'main-spitze', 'meine-news', 'merkurist', 'moeckern24', 'mpg', 'nationalgeographic', 'osna-live', 'paymentandbanking',
                   'pnn', 'prad', 'pta-in-love', 'pv-magazine', 'rblive', 'redspa', 'reporter-ohne-grenzen', 'rosalux', 'sid', 'sifa-sibe',
                   'svz', 'traunsteiner-tagblatt', 'uebermedien']

df = df[df['source'].isin(allowed_sources)]

In [29]:
df.shape

(37782, 4)

In [30]:
df.to_csv('/Users/reppmazc/Documents/IRONHACK/quests/final_project/cleaned_articles_wo_date.csv')

In [31]:
# get datetime data from gdelt rows matched based on content of 'DocumentIdentifier' save in new datetime column 
# Set up Google Cloud credentials and BigQuery client
df = pd.read_csv('') # @Janos add the path to the csv file i airdropped you yesterday

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "xx"
client = bigquery.Client()

def get_datetime(url, current_datetime):
    query = """
    SELECT DATE
    FROM `fourth-epigram-440716-f1.news_analysis.gdelt`
    WHERE DocumentIdentifier = @url
    LIMIT 1
    """
    # Use parameterized query to prevent SQL injection
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("url", "STRING", url)])
    # Run the query and fetch results
    result = client.query(query, job_config=job_config).to_dataframe()
    
    # Return the date if found, otherwise keep the existing datetime
    return result['DATE'].iloc[0] if not result.empty else current_datetime

# Update the 'datetime' column in place, preserving existing values where URLs are not found
df['datetime'] = df.apply(lambda row: get_datetime(row['url'], row['datetime']), axis=1)

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: unbilled.analysis, message: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 4937227c-6aa4-445c-ad44-f2ea9fd5731b


In [ ]:
df.to_csv('/Users/reppmazc/Documents/IRONHACK/quests/final_project/cleaned_articles_w_date.csv')

In [ ]:
df['source'].value_counts()

In [ ]:
df.shape